# 兑现模式下的特征工程

In [ ]:
# set paths
from paths import DATA_DIR, TRAIN_DATA_DIR

In [1]:
# 导入所需库
import pandas as pd
import random
import json
import psycopg2
from itertools import product
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## 1. 数据获取
### 1.1 获取兑现样本数据

In [2]:
cash_data = pd.read_csv(DATA_DIR / "115-118_congress_data.csv")
cash_members = pd.read_csv(DATA_DIR / "议员选区数据.csv")

In [3]:
def filter_cash_data_by_members(cash_data, cash_members):
    """
    根据cash_members中的数据过滤cash_data，只保留有效的议员-国会届次组合
    
    参数:
    cash_data: 包含'congress', 'mid'等列的DataFrame
    cash_members: 包含'bioguide_id', 'congress'等列的DataFrame
    
    返回:
    过滤后的cash_data DataFrame
    """
    print("=== 兑现数据过滤分析 ===")
    print(f"原始cash_data规模: {cash_data.shape}")
    print(f"cash_members规模: {cash_members.shape}")
    
    # 1. 创建有效的议员-国会届次组合集合
    valid_combinations = set(zip(cash_members['bioguide_id'], cash_members['congress']))
    print(f"有效的议员-国会届次组合数: {len(valid_combinations)}")
    
    # 2. 为cash_data创建对应的组合
    cash_data_combinations = set(zip(cash_data['mid'], cash_data['congress']))
    print(f"cash_data中的议员-国会届次组合数: {len(cash_data_combinations)}")
    
    # 3. 找出无效的组合
    invalid_combinations = cash_data_combinations - valid_combinations
    print(f"无效的议员-国会届次组合数: {len(invalid_combinations)}")
    
    # 4. 创建过滤条件
    filter_condition = cash_data.apply(
        lambda row: (row['mid'], row['congress']) in valid_combinations, 
        axis=1
    )
    
    # 5. 应用过滤
    filtered_data = cash_data[filter_condition].copy()
    
    print(f"过滤后cash_data规模: {filtered_data.shape}")
    print(f"删除的记录数: {len(cash_data) - len(filtered_data)}")
    print(f"保留率: {len(filtered_data) / len(cash_data) * 100:.2f}%")
    
    # # 6. 详细分析被过滤的数据
    # if len(cash_data) > len(filtered_data):
    #     removed_data = cash_data[~filter_condition]
    #     print(f"\n=== 被过滤数据分析 ===")
    #     print("按国会届次分布:")
    #     print(removed_data['congress'].value_counts().sort_index())
    #     print("\n按议员ID分布(前10):")
    #     print(removed_data['mid'].value_counts().head(10))
    
    return filtered_data

# 应用过滤函数
cash_data_filtered = filter_cash_data_by_members(cash_data, cash_members)

# # 验证过滤结果
# print(f"\n=== 过滤结果验证 ===")
# print("过滤前cash_data的congress分布:")
# print(cash_data['congress'].value_counts().sort_index())
# print("\n过滤后cash_data的congress分布:")
# print(cash_data_filtered['congress'].value_counts().sort_index())

# 更新cash_data变量
cash_data = cash_data_filtered

=== 兑现数据过滤分析 ===
原始cash_data规模: (154651, 6)
cash_members规模: (15251, 5)
有效的议员-国会届次组合数: 15236
cash_data中的议员-国会届次组合数: 2594
无效的议员-国会届次组合数: 0
过滤后cash_data规模: (154651, 6)
删除的记录数: 0
保留率: 100.00%


In [4]:
cash_data.head(3)

,congress,introduced_date,mid,issue,year,month
0,116,2019-05-15,D000216,Agriculture and Food,2019,5
1,119,2025-03-03,H001085,Agriculture and Food,2025,3
2,119,2025-03-03,S001220,Agriculture and Food,2025,3


## 2. 构造正负样本
### 2.1 加载数据

In [5]:
cash_data['congress'].value_counts()

congress
117    46283
118    45920
116    40213
115    14162
119     8073
Name: count, dtype: int64

### 2.2 创建样本生成函数

In [6]:
def create_samples_from_cash_data_v6(cash_data):
    """
    基于cash_data构建正负样本 (修改版v6)
    
    正样本：cash_data中每个议员在每届congress中选择的议题（去重并统计次数）
    负样本：针对每一届congress，议员没有选择的议题
    
    参数:
    cash_data: DataFrame，包含congress, year, month, mid, issue列
    
    返回:
    train_df: 包含所有正负样本的DataFrame，date列存储JSON格式的详细信息
    """
    import json
    
    positive_samples = []
    negative_samples = []
    
    # 定义所有可能的议题
    all_issues = [
        'Agriculture and Food', 'Armed Forces', 'National Security',
        'Art, Culture, Religion',
        'civil rights and liberties, minority issues', 'Commerce',
        'Congressional political operations', 'Crime and Law enforcement',
        'economics and public finance', 'education',
        'emergency management', 'energy', 'environmental protection',
        'finance and financial sector', 'international trade',
        'international finance', 'government operations and politics',
        'health', 'Housing and Community Development', 'immigration',
        'alliance and collective security', 'human rights',
        'foreign affairs', 'labor and employment', 'law',
        'public lands and natural resources',
        'science, technology, communications',
        'social science and history', 'social welfare', 'taxation',
        'transportation and public works', 'water resources development'
    ]
    
    print(f"议题总数: {len(all_issues)}")
    
    # 1. 按congress分组处理
    congress_groups = cash_data.groupby('congress')
    
    for congress, group in congress_groups:
        print(f"\n处理第 {congress} 届congress...")
        
        # 按议员分组，获取每个议员在该届congress中选择的议题信息
        member_groups = group.groupby('mid')
        
        congress_positive = 0
        congress_negative = 0
        
        for mid, member_data in member_groups:
            # 2. 统计该议员在该届congress中每个议题的选择情况
            issue_stats = {}
            
            # 按议题分组，统计次数和收集日期信息
            issue_groups = member_data.groupby('issue')
            for issue, issue_data in issue_groups:
                # 统计该议题的选择次数
                count = len(issue_data)
                
                # 收集所有相关的日期信息
                dates_info = []
                for _, row in issue_data.iterrows():
                    dates_info.append({
                        'year': int(row['year']),
                        'month': int(row['month'])
                    })
                
                # 存储该议题的完整信息
                issue_stats[issue] = {
                    'count': count,
                    'dates': dates_info,
                    'first_date': f"{dates_info[0]['year']}-{dates_info[0]['month']:02d}",
                    'congress': int(congress)
                }
            
            # 3. 构建正样本：该议员选择的每个议题
            for issue, stats in issue_stats.items():
                # 将统计信息转换为JSON字符串
                date_json = json.dumps(stats, ensure_ascii=False)
                
                positive_samples.append([
                    congress,
                    issue,
                    mid,
                    1,  # 正样本标签
                    date_json  # JSON格式的日期和统计信息
                ])
                congress_positive += 1
            
            # 4. 构建负样本：该议员未选择的议题
            selected_issues = set(issue_stats.keys())
            unselected_issues = set(all_issues) - selected_issues
            
            for issue in unselected_issues:
                # 负样本的JSON信息（没有实际选择记录）
                negative_info = {
                    'count': 0,
                    'dates': [],
                    'first_date': None,
                    'congress': int(congress)
                }
                date_json = json.dumps(negative_info, ensure_ascii=False)
                
                negative_samples.append([
                    congress,
                    issue,
                    mid,
                    0,  # 负样本标签
                    date_json
                ])
                congress_negative += 1
        
        print(f"第 {congress} 届congress - 正样本: {congress_positive}, 负样本: {congress_negative}")
    
    print(f"\n=== 总体统计 ===")
    print(f"总正样本数量: {len(positive_samples)}")
    print(f"总负样本数量: {len(negative_samples)}")
    print(f"总样本数量: {len(positive_samples) + len(negative_samples)}")
    print(f"正负样本比例: 1:{len(negative_samples)/len(positive_samples):.1f}")
    
    # 转换为DataFrame
    columns = ['congress', 'issue', 'mid', 'label', 'date']
    positive_df = pd.DataFrame(positive_samples, columns=columns)
    negative_df = pd.DataFrame(negative_samples, columns=columns)
    
    # 合并所有样本
    train_df = pd.concat([positive_df, negative_df], ignore_index=True)
    
    # 随机打乱数据
    train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    return train_df

In [7]:
# # 使用新版本函数
# cash_data = pd.read_csv("./input/115-118_congress_data.csv")

# print(f"Cash data 原始规模: {len(cash_data)} 行")
# print(f"时间范围: {cash_data['year'].min()}-{cash_data['year'].max()}")
# print(f"议员数量: {cash_data['mid'].nunique()}")
# print(f"议题数量: {cash_data['issue'].nunique()}")

# # 创建完整的训练数据集（JSON格式）
# train_df_v6 = create_samples_from_ca
# sh_data_v6(cash_data)

# print(f"\n最终训练数据规模: {train_df_v6.shape}")
# print(f"标签分布:\n{train_df_v6['label'].value_counts()}")

# # 验证去重效果
# print(f"\n验证去重效果:")
# duplicate_check = train_df_v6.groupby(['congress', 'mid', 'issue']).size()
# duplicates = duplicate_check[duplicate_check > 1]
# print(f"重复的(congress, mid, issue)组合数量: {len(duplicates)}")

# # 显示数据结构
# print(f"\n数据结构预览:")
# print(train_df_v6.head())
# print(f"\n列名: {list(train_df_v6.columns)}")


# # 保存完整的训练数据
# train_df_v6.to_csv("./output/cash_data_label.csv", index=False)

In [8]:
train_df_v6 = pd.read_csv(TRAIN_DATA_DIR / "cash_data_label.csv")

### 2.3 生成所有样本并平衡正负样本比例

In [9]:
# # 计算需要采样的负样本数量
# num_positive_samples = len(positive_samples_df)
# num_negative_samples_to_sample = num_positive_samples * 2

# # 从负样本中随机采样（有放回）
# negative_samples_df = negative_samples_df.sample(n=num_negative_samples_to_sample, random_state=42, replace=True)

# # 合并正负样本作为训练集
# train_df = pd.concat([negative_samples_df, positive_samples_df])

# # # 处理日期和国会届次
# # # 格式化日期并转换为datetime类型
# train_df["date"] = train_df.apply(lambda x: f'{x["cur_year"]}-{x["cur_month"]:02d}', axis=1)
# train_df["date"] = pd.to_datetime(train_df["date"], format='%Y-%m')

In [10]:
# 使用范围筛选115-118届次的数据
train_df = train_df_v6[(train_df_v6['congress'] >= 115) & (train_df_v6['congress'] <= 118)]

In [11]:
train_df['label'].value_counts()

label
1    42879
0    25761
Name: count, dtype: int64

In [12]:
train_df.shape

(68640, 5)

In [13]:
# train_df[train_df['mid']=='G000584']

## 3. 从PostgreSQL获取议员和委员会数据
### 3.1 从数据库获取议员和委员会数据

In [ ]:
connection = psycopg2.connect(
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    database=os.getenv("PG_DB")
)

cursor = connection.cursor()

In [15]:
# 获取目标议员的唯一标识列表
target_mid_list = list(train_df["mid"].unique())

# 将列表转换为逗号分隔的字符串，并用单引号包裹每个元素
formatted_mid_list = ', '.join(f"'{mid}'" for mid in target_mid_list)

# 构建查询字符串，直接在 SQL 中处理数据
query = f"""
    SELECT 
        bioguide_id, 
        full_name, 
        chamber, 
        CASE 
            WHEN birth_year IS NOT NULL THEN 2024 - birth_year 
            ELSE NULL 
        END AS age, 
        party, 
        state_code, 
        district, 
        CASE 
            WHEN name_info::jsonb ? 'honor_name' THEN name_info::jsonb->>'honor_name' 
            ELSE NULL 
        END AS honor,
        terms
    FROM 
        members
    WHERE 
        bioguide_id IN ({formatted_mid_list})
"""

# 执行查询并获取数据
cursor.execute(query)
members_data = cursor.fetchall()

# ---------------------------------------------------------------------------------------------------------
# 从 committees_membership 和 committees 数据库中获取所有数据
# 先创建临时表 committees_official，将 committees 中 official_name 匹配到 committees_membership
# 然后根据 committees_membership 完成数据查询
query_combined = f"""

    WITH committees_official AS(
    SELECT 
		ch.thomas_id,
		ch.thomas_num,
		ch.congress,
		c.official_name
    FROM 
		committees_history ch
    LEFT JOIN 
		committees c
    ON 
		c.thomas_id = ch.thomas_id AND c.thomas_num = ch.thomas_num    
    )
    SELECT 
        cm.member_id, 
        cm.congress, 
        cm.thomas_id AS cm_thomas_id, 
        cm.thomas_num, 
        cm.title, 
        co.official_name
    FROM 
        committees_membership cm
    LEFT JOIN 
        committees_official co
    ON 
        cm.thomas_id = co.thomas_id AND cm.thomas_num = co.thomas_num AND cm.congress = co.congress
    WHERE
        cm.member_id IN ({formatted_mid_list}) AND cm.thomas_num = '00' AND cm.congress >= 115
"""
cursor.execute(query_combined)
combined_data = cursor.fetchall()

# ---------------------------------------------------------------------------------------------------------
# 获取选区的数据
sql = """
    SELECT congress, state_code, district, ppvi, npvi, employee, payroll, ethnicity
    FROM district
    WHERE congress >= 115;
"""
cursor.execute(sql)
district_data = cursor.fetchall()

# ---------------------------------------------------------------------------------------------------------
# 获取州的数据
sql = """
    SELECT congress, state_code, ppvi AS state_ppvi, npvi AS state_npvi, employee AS state_employee, payroll AS state_payroll, ethnicity AS state_ethnicity
    FROM state
    WHERE congress >= 115;
"""
cursor.execute(sql)
state_data = cursor.fetchall()

# ---------------------------------------------------------------------------------------------------------
# 构建意识形态的数据
sql = f"""
SELECT 
        cm.member_id, 
        cm.congress, 
        cm.thomas_id AS cm_thomas_id, 
        cm.thomas_num, 
        co.ideology
    FROM 
        committees_membership cm
    LEFT JOIN 
        committees_analysis co
    ON 
        cm.thomas_id = co.thomas_id AND cm.thomas_num = co.thomas_num AND cm.congress = co.congress
    WHERE
        cm.member_id IN ({formatted_mid_list}) AND cm.thomas_num = '00' AND cm.congress >= 115
"""
cursor.execute(sql)
df_ideology = cursor.fetchall()

In [16]:
# 关闭数据库连接
cursor.close()
connection.close()

### 3.2 处理获取的数据

In [17]:
# 将数据转换为 DataFrame
columns = ["bioguide_id", "full_name", "chamber", "age", "party", "state_code", "district", "honor","terms"]
df_members = pd.DataFrame(members_data, columns=columns)

columns = ["member_id", "congress", "cm_thomas_id", "thomas_num", "title", "official_name"]
df_combined = pd.DataFrame(combined_data, columns=columns)

columns = ["congress", "state_code", "district", "ppvi", "npvi", "employ", "payroll", "ethnicity"]
df_district = pd.DataFrame(district_data, columns=columns)

columns = ["congress", "state_code", "state_ppvi", "state_npvi", "state_employ", "state_payroll", "state_ethnicity"]
df_state = pd.DataFrame(state_data, columns=columns)

columns = ["member_id", "congress", "cm_thomas_id", "thomas_num", "ideology"]
df_ideology = pd.DataFrame(df_ideology, columns=columns)

In [18]:
# 更新PVI的取值
# === 用PVI表更新df_district中的ppvi和npvi ===
import numpy as np

# 读取PVI数据
df_pvi = pd.read_excel('./input/pvi-full-new.xlsx', sheet_name='full')
# 提取PVI中的字母部分为ppvi，数字部分为npvi
df_pvi['ppvi'] = df_pvi['PVI'].str.extract(r'([A-Za-z]+)')
df_pvi['npvi'] = df_pvi['PVI'].str.extract(r'(\d+)')

# 2. 仅保留df_pvi中用于匹配和更新的必要列，使用copy()避免警告
df_pvi_sub = df_pvi[['congress', 'postal', 'district', 'ppvi', 'npvi']].copy()

# === 过滤掉congress为NaN的数据 ===
print(f"过滤前PVI数据规模: {df_pvi_sub.shape}")
print(f"congress列缺失值数量: {df_pvi_sub['congress'].isna().sum()}")

# 删除congress为NaN的行
df_pvi_sub = df_pvi_sub.dropna(subset=['congress'])

print(f"过滤后选区级PVI数据规模: {df_pvi_sub.shape}")
print(f"包含的congress: {sorted(df_pvi_sub['congress'].unique())}")

# === 统一字符格式，处理匹配键 ===
# 1. 统一congress列的数据类型
df_pvi_sub['congress'] = df_pvi_sub['congress'].astype(int)
df_district['congress'] = df_district['congress'].astype(int)

# 2. 统一州代码格式：去除空格，转为大写
df_pvi_sub['postal'] = df_pvi_sub['postal'].astype(str).str.strip().str.upper()
df_district['state_code'] = df_district['state_code'].astype(str).str.strip().str.upper()

# 3. 统一district格式：转为字符串，去除空格
df_pvi_sub['district'] = df_pvi_sub['district'].astype(str).str.strip()
df_district['district'] = df_district['district'].astype(str).str.strip()

# 输出格式统一后的匹配键信息
print(f"\n=== 格式统一后的匹配键信息 ===")
print(f"df_pvi_sub - congress类型: {df_pvi_sub['congress'].dtype}, postal类型: {df_pvi_sub['postal'].dtype}, district类型: {df_pvi_sub['district'].dtype}")
print(f"df_district - congress类型: {df_district['congress'].dtype}, state_code类型: {df_district['state_code'].dtype}, district类型: {df_district['district'].dtype}")

# 1. 先将df_district中的ppvi和npvi设为NaN，方便后续整体替换
df_district = df_district.copy()  # 创建副本避免警告
df_district['ppvi'] = np.nan
df_district['npvi'] = np.nan

# 3. 按congress、state_code/postal、district三列合并，获取最新ppvi和npvi
df_district = df_district.merge(
    df_pvi_sub,
    left_on=['congress', 'state_code', 'district'],
    right_on=['congress', 'postal', 'district'],
    how='left',
    suffixes=('', '_new')
)

# 4. 用合并后的新ppvi和npvi覆盖原有值
df_district['ppvi'] = df_district['ppvi_new']
df_district['npvi'] = df_district['npvi_new']

# 5. 删除合并产生的多余列
df_district = df_district.drop(columns=['ppvi_new', 'npvi_new', 'postal'])

print(f"更新后df_district的规模: {df_district.shape}")
print(f"ppvi缺失值数量: {df_district['ppvi'].isna().sum()}")
print(f"npvi缺失值数量: {df_district['npvi'].isna().sum()}")

过滤前PVI数据规模: (2970, 5)
congress列缺失值数量: 0
过滤后选区级PVI数据规模: (2970, 5)
包含的congress: [113, 114, 115, 116, 117, 118]

=== 格式统一后的匹配键信息 ===
df_pvi_sub - congress类型: int32, postal类型: object, district类型: object
df_district - congress类型: int32, state_code类型: object, district类型: object
更新后df_district的规模: (1748, 8)
ppvi缺失值数量: 35
npvi缺失值数量: 71


In [19]:
# 获取所有唯一的state_code
unique_state_codes = df_state['state_code'].unique()

# 检查是否已经存在115届数据，避免重复添加
if 115 not in df_state['congress'].values:
    # 为115届创建空数据行
    df_115_rows = []
    for state_code in unique_state_codes:
        df_115_rows.append({'congress': 115, 'state_code': state_code})
    
    # 转为DataFrame并合并
    df_115 = pd.DataFrame(df_115_rows)
    df_state = pd.concat([df_state, df_115], ignore_index=True)
    print(f"已添加115届数据")
else:
    print(f"115届数据已存在，跳过添加")

print(f"添加后规模: {len(df_state)}")
print(f"各届次分布:\n{df_state['congress'].value_counts().sort_index()}")

已添加115届数据
添加后规模: 208
各届次分布:
congress
115    52
116    52
117    52
118    52
Name: count, dtype: int64


In [20]:
# 州级PVI数据更新
df_pvi_state = pd.read_excel('./input/pvi-full-new.xlsx', sheet_name='full')

df_pvi_state['ppvi'] = df_pvi_state['PVI'].str.extract(r'([A-Za-z]+)')
df_pvi_state['npvi'] = df_pvi_state['PVI'].str.extract(r'(\d+)')

# 筛选district为'99'的数据作为州级数据
df_pvi_state_sub = df_pvi_state[df_pvi_state['district'] == 99][['congress', 'postal', 'ppvi', 'npvi']]

print(f"州级PVI数据规模: {df_pvi_state_sub.shape}")
print(f"包含的congress: {sorted(df_pvi_state_sub['congress'].unique())}")

# 1. 先将df_state中的state_ppvi和state_npvi设为NaN，方便后续整体替换
df_state['state_ppvi'] = np.nan
df_state['state_npvi'] = np.nan

# 2. 按congress、state_code/postal两列合并，获取最新的州级ppvi和npvi
df_state = df_state.merge(
    df_pvi_state_sub,
    left_on=['congress', 'state_code'],
    right_on=['congress', 'postal'],
    how='left',
    suffixes=('', '_new')
)

# 3. 用合并后的新ppvi和npvi覆盖原有的state_ppvi和state_npvi值
df_state['state_ppvi'] = df_state['ppvi']
df_state['state_npvi'] = df_state['npvi']

# 4. 删除合并产生的多余列
df_state = df_state.drop(columns=['ppvi', 'npvi', 'postal'])

print(f"更新后df_state的规模: {df_state.shape}")
print(f"state_ppvi缺失值数量: {df_state['state_ppvi'].isna().sum()}")
print(f"state_npvi缺失值数量: {df_state['state_npvi'].isna().sum()}")

州级PVI数据规模: (336, 4)
包含的congress: [113, 114, 115, 116, 117, 118]
更新后df_state的规模: (208, 7)
state_ppvi缺失值数量: 4
state_npvi缺失值数量: 12


In [21]:
# 填充df_member缺失的district值
# 对于Senate成员，将district设为'99'
# 对于House成员，将district设为'At Large'

# 创建填充条件
senate_mask = (df_members['district'].isna()) & (df_members['chamber'] == 'Senate')
house_mask = (df_members['district'].isna()) & (df_members['chamber'] == 'House of Representatives')

# 打印填充前的统计
print(f"填充前Senate缺失district数: {senate_mask.sum()}")
print(f"填充前House缺失district数: {house_mask.sum()}")

# 根据条件填充值
# df_members.loc[senate_mask, 'district'] = 'At Large'
# df_members.loc[house_mask, 'district'] = '99'

df_members.loc[senate_mask, 'district'] = '99'
df_members.loc[house_mask, 'district'] = 'At Large'


# 打印填充后的统计
print(f"填充后district列总缺失数: {df_members['district'].isna().sum()}")

填充前Senate缺失district数: 138
填充前House缺失district数: 16
填充后district列总缺失数: 0


In [22]:
# 合并df_ideology中的ideology列到df_combined中
df_combined = df_combined.merge(
    df_ideology[['member_id', 'congress', 'cm_thomas_id', 'thomas_num', 'ideology']],
    on=['member_id', 'congress', 'cm_thomas_id', 'thomas_num'],
    how='left'
)

In [23]:
# 根据议员称呼确定性别 (Mr. 为男性，其他为女性)
df_members["gender"] = df_members["honor"].apply(lambda x: 1 if x == "Mr." else 0)

### 3.3 合并基础特征

In [24]:
# 合并议员信息
train_df = train_df.merge(
    df_members, 
    left_on=["mid"], 
    right_on=["bioguide_id"], 
    how="left"
)
print(f"合并议员信息后的数据规模: {train_df.shape}")

合并议员信息后的数据规模: (68640, 15)


In [25]:
# train_df[train_df['district'] == '99'].head(3)

In [26]:
train_df_beifen = train_df.copy()
train_df = train_df_beifen.copy()

In [27]:
train_df[train_df['mid'] == 'R000103'][['congress','district']].head(2)

,congress,district
1147,118,2
1672,117,2


In [28]:
train_df['district'] = 99999

In [29]:
# 选区数据更新
df_members_district_origin = pd.read_csv('./input/议员选区数据.csv')

# 筛选出115-118届次的数据
df_members_district = df_members_district_origin[(df_members_district_origin['congress'] >= 115) & (df_members_district_origin['congress'] <= 118)]

# 使用df_members_district更新train_df中的district数据
print(f"=== 更新前统计 ===")
print(f"train_df规模: {train_df.shape}")
print(f"df_members_district规模: {df_members_district.shape}")
print(f"train_df中district缺失值数量: {train_df['district'].isna().sum()}")

# 检查匹配键的数据类型和格式
print(f"\n=== 数据类型检查 ===")
print(f"train_df - mid类型: {train_df['mid'].dtype}, congress类型: {train_df['congress'].dtype}")
print(f"df_members_district - bioguide_id类型: {df_members_district['bioguide_id'].dtype}, congress类型: {df_members_district['congress'].dtype}")
print(f"train_df - district类型: {train_df['district'].dtype}")
print(f"df_members_district - district类型: {df_members_district['district'].dtype}")

# 统一数据类型
train_df['congress'] = train_df['congress'].astype(int)
df_members_district['congress'] = df_members_district['congress'].astype(int)

# # 特别处理df_members_district的district列：先转为整数再转为字符串，避免小数点
# # 对于float类型的district，先填充NaN，然后转为int再转为str
df_members_district['district'] = df_members_district['district'].fillna(-999)  # 临时填充
df_members_district['district'] = df_members_district['district'].astype(int)
df_members_district['district'] = df_members_district['district'].astype(str)
df_members_district['district'] = df_members_district['district'].replace('-999', pd.NA)  # 恢复NaN

# 更新前清空
train_df['district'] = 99999

# 统一train_df的district列数据类型为字符串
train_df['district'] = train_df['district'].astype(str)


# 处理可能的空值，将字符串'nan'转换为pandas的NA
train_df['district'] = train_df['district'].replace('nan', pd.NA)
df_members_district['district'] = df_members_district['district'].replace('nan', pd.NA)

# 创建筛选条件：删除K000394和S001150议员中district为空的数据
condition_to_remove = (
    df_members_district['bioguide_id'].isin(['K000394', 'S001150']) & 
    (df_members_district['district'].isna() | 
     (df_members_district['district'] == '') | 
     (df_members_district['district'] == 'nan'))
)

# 保留不满足删除条件的数据
df_members_district = df_members_district[~condition_to_remove]
print(f"处理后df_members_district规模: {df_members_district.shape}")
print(f"删除了 {condition_to_remove.sum()} 行特定议员的district为空数据")



# 执行左连接合并，更新district信息
train_df_updated = train_df.merge(
    df_members_district[['bioguide_id', 'congress', 'district']],
    left_on=['mid', 'congress'],
    right_on=['bioguide_id', 'congress'],
    how='left',
    suffixes=('', '_new')
)

# 用新的district值覆盖原有值（优先使用df_members_district中的数据）
# 如果df_members_district中有匹配的数据，就使用新值；否则保持原值
train_df_updated['district'] = train_df_updated['district_new'].fillna(train_df_updated['district'])

# 确保最终的district列保持字符串类型
train_df_updated['district'] = train_df_updated['district'].astype(str)

# 再次处理可能出现的'nan'字符串
train_df_updated['district'] = train_df_updated['district'].replace('nan', pd.NA)

# 删除合并产生的多余列
train_df_updated = train_df_updated.drop(columns=['district_new', 'bioguide_id'])

# 更新原始的train_df
train_df = train_df_updated

print(f"\n=== 更新后统计 ===")
print(f"更新后train_df规模: {train_df.shape}")
print(f"更新后district缺失值数量: {train_df['district'].isna().sum()}")
print(f"更新后district数据类型: {train_df['district'].dtype}")

# 统计匹配效果
matched_count = len(df_members_district.merge(
    train_df[['mid', 'congress']].drop_duplicates(),
    left_on=['bioguide_id', 'congress'],
    right_on=['mid', 'congress'],
    how='inner'
))

print(f"成功匹配的记录数: {matched_count}")

# 检查district值的样例，确认没有小数点
print(f"\n=== District值样例检查 ===")
print(f"District唯一值示例: {sorted(train_df['district'].dropna().unique())[:10]}")

=== 更新前统计 ===
train_df规模: (68640, 15)
df_members_district规模: (2221, 5)
train_df中district缺失值数量: 0

=== 数据类型检查 ===
train_df - mid类型: object, congress类型: int64
df_members_district - bioguide_id类型: object, congress类型: int64
train_df - district类型: int64
df_members_district - district类型: float64
处理后df_members_district规模: (2219, 5)
删除了 2 行特定议员的district为空数据

=== 更新后统计 ===
更新后train_df规模: (68640, 15)
更新后district缺失值数量: 0
更新后district数据类型: object
成功匹配的记录数: 2145

=== District值样例检查 ===
District唯一值示例: ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18']


In [30]:
# train_df[train_df['mid'] == 'R000103'].head(2)

In [31]:
# train_df['district']

In [32]:
# 填充df_member缺失的district值
# 对于Senate成员，将district设为'99'
# 对于House成员，将district设为'At Large'

# 创建填充条件
senate_mask = (train_df['district'] == '99999') & (train_df['chamber'] == 'Senate')
house_mask = (train_df['district'] == '99999') & (train_df['chamber'] == 'House of Representatives')

# 打印填充前的统计
print(f"填充前Senate缺失district数: {senate_mask.sum()}")
print(f"填充前House缺失district数: {house_mask.sum()}")

# 根据条件填充值
# train_df.loc[senate_mask, 'district'] = 'At Large'
# train_df.loc[house_mask, 'district'] = '99'

train_df.loc[senate_mask, 'district'] = '99'
train_df.loc[house_mask, 'district'] = 'At Large'

# 打印填充后的统计
print(f"填充后district列总缺失数: {train_df['district'].isna().sum()}")

填充前Senate缺失district数: 13024
填充前House缺失district数: 1280
填充后district列总缺失数: 0


In [33]:
train_df[train_df['district'] == '35'].head(3)

,congress,issue,mid,label,date,full_name,chamber,age,party,state_code,district,honor,terms,gender,bioguide_id_new
223,118,social welfare,T000474,1,"{""count"": 1, ""dates"": [{""year"": 2023, ""month"":...","Rep. Torres, Norma J. [D-CA-35]",House of Representatives,59,Democratic,CA,35,Mrs.,"[{'chamber': 'House of Representatives', 'endY...",0,T000474
392,115,international finance,T000474,0,"{""count"": 0, ""dates"": [], ""first_date"": null, ...","Rep. Torres, Norma J. [D-CA-35]",House of Representatives,59,Democratic,CA,35,Mrs.,"[{'chamber': 'House of Representatives', 'endY...",0,T000474
534,118,health,C001131,1,"{""count"": 2, ""dates"": [{""year"": 2024, ""month"":...","Rep. Casar, Greg [D-TX-35]",House of Representatives,35,Democratic,TX,35,Mr.,"[{'chamber': 'House of Representatives', 'endY...",1,C001131


In [34]:
# train_df[
# (train_df['full_name'] == 'Rep. Doggett, Lloyd [D-TX-37]') &
#     (train_df['congress'] == 118)
# ]

In [35]:
# # 修正后的查询条件
# train_df[
# (train_df['congress'] == 117) &
# (train_df['state_code'] == 'TX') & 
# (train_df['district'] == 35)  # 将数字35改为字符串'35'
# ]

In [36]:
def calculate_seniority(terms_data, current_congress):
    """
    计算议员的资历(seniority)，基于terms列表中的任期信息
    只计算严格小于current_congress的任期（不含当前届次）
    
    参数:
    terms_data: JSON字符串或Python对象(列表/字典)
    current_congress: 当前国会届次，只计算此届次之前的资历
    
    返回:
    整数: 议员截止当前届次前的总在任年数
    """
    import json
    
    # 如果terms_data是字符串，尝试解析为Python对象
    if isinstance(terms_data, str):
        try:
            terms_list = json.loads(terms_data)
        except json.JSONDecodeError:
            return 0  # 返回0，表示无法解析
    else:
        terms_list = terms_data
    
    # 确保terms_list是一个列表
    if not isinstance(terms_list, list):
        return 0
    
    # 计算总在任时间
    total_years = 0
    for term in terms_list:
        term_congress = term.get('congress')
        
        # 跳过未来的任期或大于等于当前国会届次的任期
        if term_congress is None or term_congress >= current_congress:
            continue
            
        # 计算当前任期的年数
        start_year = term.get('startYear')
        end_year = term.get('endYear')
        
        # 确保startYear和endYear都存在且为数字
        if isinstance(start_year, (int, float)) and isinstance(end_year, (int, float)):
            term_duration = end_year - start_year
            total_years += term_duration
            
    return total_years

In [37]:
# 应用函数到train_df，根据每行的congress计算相应的资历
train_df['seniority'] = train_df.apply(
    lambda row: calculate_seniority(row['terms'], row['congress']), 
    axis=1
)

# # 查看结果
# print("添加资历特征后:")
# print(train_df[['mid', 'congress', 'seniority']].head())

In [38]:
# 议员对应委员会去重，会出现一个议员同一届加入了多届委员会的情况；
# 优先使用有 Title 的委员会，没有则取第一个委员会
df_combined.drop_duplicates(subset=["member_id", "congress"])

def filter_committees(df):
    filtered_rows = []
    grouped = df.groupby(['member_id', 'congress'])

    for name, group in grouped:
        if group['title'].notna().any():
            filtered_row = group[group['title'].notna()].iloc[0]
        else:
            filtered_row = group.iloc[0]
        filtered_rows.append(filtered_row)

    return pd.DataFrame(filtered_rows)

df_combined = filter_committees(df_combined)

In [39]:
# 合并委员会信息
train_df = train_df.merge(
    df_combined,
    left_on=["mid", "congress"],
    right_on=["member_id", "congress"],
    how="left"
)

# 重命名相关列
train_df = train_df.rename(columns={
    "official_name": "committee_el",
    "title": "com_post"
})
print(f"合并委员会信息后的数据规模: {train_df.shape}")

合并委员会信息后的数据规模: (68640, 22)


In [40]:
# # 展开特征列表的函数
# def expand_list_column(df, column_name):
#     """将列表类型的列展开为多个列"""
#     expanded_df = pd.DataFrame(df[column_name].tolist(), index=df.index)
#     expanded_df.columns = [f"{column_name}{i + 1}" for i in range(expanded_df.shape[1])]
#     return expanded_df


# 展开特征列表的函数（改进版）
def expand_list_column(df, column_name):
    """将列表类型的列展开为多个列，处理空值情况"""
    # 处理空值，将NaN替换为空列表
    column_data = df[column_name].fillna('').apply(lambda x: x if isinstance(x, list) else [])
    
    # 如果所有值都是空列表，创建一个包含单列的DataFrame
    if all(len(item) == 0 for item in column_data):
        expanded_df = pd.DataFrame({f"{column_name}1": [None] * len(df)}, index=df.index)
    else:
        # 找出最大长度
        max_length = max(len(item) for item in column_data if len(item) > 0)
        # 将所有列表填充到相同长度
        padded_data = [item + [None] * (max_length - len(item)) for item in column_data]
        expanded_df = pd.DataFrame(padded_data, index=df.index)
        expanded_df.columns = [f"{column_name}{i + 1}" for i in range(expanded_df.shape[1])]
    
    return expanded_df


# 扩展 district 中的 employ, payroll, and ethnicity 列
employ_expanded = expand_list_column(df_district, "employ")
payroll_expanded = expand_list_column(df_district, "payroll")
ethnicity_expanded = expand_list_column(df_district, "ethnicity")

df_district_expanded = pd.concat([
    df_district.drop(columns=["employ", "payroll", "ethnicity"]), 
    employ_expanded, 
    payroll_expanded, 
    ethnicity_expanded
], axis=1)

# 扩展 state 中的 employ, payroll, and ethnicity 列
state_employ_expanded = expand_list_column(df_state, "state_employ")
state_payroll_expanded = expand_list_column(df_state, "state_payroll")
state_ethnicity_expanded = expand_list_column(df_state, "state_ethnicity")

df_state_expanded = pd.concat([
    df_state.drop(columns=["state_employ", "state_payroll", "state_ethnicity"]),
    state_employ_expanded,
    state_payroll_expanded,
    state_ethnicity_expanded
], axis=1)

In [41]:
# 从本地加载115届州数据，合并到现有的数据中
df_state_115 = pd.read_csv(DATA_DIR / 'state_115_1220_with_codes.csv')

In [42]:
# 准备df_state_115的数据，重命名列以匹配目标格式
df_state_115_prepared = df_state_115.copy()

# 重命名列以匹配df_state_expanded的命名规则
df_state_115_prepared = df_state_115_prepared.rename(columns={
    'Congress': 'congress',  # 统一congress列名
    'employ1': 'state_employ1',
    'employ2': 'state_employ2', 
    'employ3': 'state_employ3',
    'payroll1': 'state_payroll1',
    'payroll2': 'state_payroll2',
    'payroll3': 'state_payroll3',
    'ethnicity1': 'state_ethnicity1',
    'ethinicity2': 'state_ethnicity2', 
    'ethinicity3': 'state_ethnicity3'  
})

# 只保留需要合并的列
merge_columns = ['congress', 'state_code', 'state_employ1', 'state_employ2', 'state_employ3',
                'state_payroll1', 'state_payroll2', 'state_payroll3', 
                'state_ethnicity1', 'state_ethnicity2', 'state_ethnicity3']

df_state_115_for_merge = df_state_115_prepared[merge_columns]

# 统一数据类型
df_state_115_for_merge['congress'] = df_state_115_for_merge['congress'].astype(int)
df_state_expanded['congress'] = df_state_expanded['congress'].astype(int)

# 统一字符串格式
df_state_115_for_merge['state_code'] = df_state_115_for_merge['state_code'].astype(str).str.strip().str.upper()
df_state_expanded['state_code'] = df_state_expanded['state_code'].astype(str).str.strip().str.upper()

print(f"合并前df_state_expanded规模: {df_state_expanded.shape}")
print(f"df_state_115_for_merge规模: {df_state_115_for_merge.shape}")

# 执行左连接合并
df_state_expanded = df_state_expanded.merge(
    df_state_115_for_merge,
    on=['congress', 'state_code'],
    how='left',
    suffixes=('', '_115')
)

# 用df_state_115的数据填充df_state_expanded中的空值
target_columns = ['state_employ1', 'state_employ2', 'state_employ3',
                 'state_payroll1', 'state_payroll2', 'state_payroll3', 
                 'state_ethnicity1', 'state_ethnicity2', 'state_ethnicity3']

for col in target_columns:
    col_115 = col + '_115'
    if col_115 in df_state_expanded.columns:
        # 用115数据填充空值
        df_state_expanded[col] = df_state_expanded[col].fillna(df_state_expanded[col_115])
        # 删除临时列
        df_state_expanded = df_state_expanded.drop(columns=[col_115])

print(f"合并后df_state_expanded规模: {df_state_expanded.shape}")

# # 检查合并效果
# for col in target_columns:
#     missing_count = df_state_expanded[col].isna().sum()
#     print(f"{col}缺失值数量: {missing_count}")

合并前df_state_expanded规模: (208, 13)
df_state_115_for_merge规模: (52, 11)
合并后df_state_expanded规模: (208, 13)


In [43]:
# 合并选区数据
train_df = train_df.merge(
    df_district_expanded, 
    on=["congress", "state_code", "district"], 
    how="left"
)

train_df.shape

(68640, 33)

In [44]:
# 合并州的数据
train_df = train_df.merge(
    df_state_expanded, 
    on=["congress", "state_code"], 
    how="left"
)
train_df.shape

(68640, 44)

In [45]:
# df_district_expanded[
# (df_district_expanded['congress'] == 117) &
# (df_district_expanded['state_code'] == 'TX') &
# (df_district_expanded['district'] == '35')
# ]

In [46]:
# # 修正后的查询条件
# train_df[
# (train_df['congress'] == 117) &
# (train_df['state_code'] == 'TX') & 
# (train_df['district'] == "35")  
# ]

对district取值为At Large和99的数据使用州层面的数据进行填充

In [47]:
def fill_with_state_data_mask(df, mask, 
                             columns=['ppvi', 'npvi', 'employ1', 'employ2', 'employ3', 
                                      'payroll1', 'payroll2', 'payroll3',
                                      'ethnicity1', 'ethnicity2', 'ethnicity3'],
                             print_detail=True, label=""):
    """
    对满足mask的行，用state_xxx列批量填充columns列的空值。
    在原df上直接修改并返回。
    """
    null_before = df.loc[mask, columns].isna().sum()

    for col in columns:
        state_col = 'state_' + col
        if state_col in df.columns:
            df.loc[mask, col] = df.loc[mask, state_col]
    
    null_after = df.loc[mask, columns].isna().sum()
    filled = null_before.sum() - null_after.sum()
    if print_detail:
        print(f"\n对{label}总共填充了 {filled} 个空值\n")
    return df

# 用法：
senate_mask = train_df['chamber'] == 'Senate'
train_df = fill_with_state_data_mask(train_df, mask=senate_mask, label="Senate成员数据")
district_99_mask = train_df['district'] == '99'
train_df = fill_with_state_data_mask(train_df, mask=district_99_mask, label="district=99的数据")


对Senate成员数据总共填充了 142688 个空值


对district=99的数据总共填充了 0 个空值



In [48]:
# train_df[train_df['mid'] == 'R000103']

### 检查缺失数据

In [49]:
train_df_view = train_df[train_df['payroll1'].isna()]
train_df_view[['chamber']].value_counts()

chamber                 
House of Representatives    480
Name: count, dtype: int64

In [50]:
# train_df[(train_df['mid']=='B001299') &
#          (train_df['congress']==117)  &
#          (train_df['cur_month']==2)  ].head(5)

## 4. 从本地文件加载其他特征
### 4.1 准备议题标识符

In [51]:
# 创建小写版本的议题名称列，用于后续合并
train_df["issue_lower"] = train_df["issue"].apply(lambda x: x.lower())

### 4.2 加载本地数据
- 政党议题偏好
- 国际压力值
- 产业卷入度数据
- 这一步才有的party_tp变量

In [52]:
df_issue = pd.read_excel(DATA_DIR / "issue-full_new.xlsx", sheet_name="Sheet1")
df_issue['issue'] = df_issue["issue"].apply(lambda x:x.lower())

In [53]:
train_df = train_df.merge(
    df_issue, 
    left_on=['issue_lower','congress'], 
    right_on=['issue','congress'], 
    how="left",
)

# # 删除不需要的重复列
if 'issue_y' in train_df.columns:
    train_df = train_df.drop(columns=['issue_y'])

# 将issue_x重命名为issue
if 'issue_x' in train_df.columns:
    train_df = train_df.rename(columns={'issue_x': 'issue'})

In [54]:
train_df.columns

Index(['congress', 'issue', 'mid', 'label', 'date', 'full_name', 'chamber',
       'age', 'party', 'state_code', 'district', 'honor', 'terms', 'gender',
       'bioguide_id_new', 'seniority', 'member_id', 'cm_thomas_id',
       'thomas_num', 'com_post', 'committee_el', 'ideology', 'ppvi', 'npvi',
       'employ1', 'employ2', 'employ3', 'payroll1', 'payroll2', 'payroll3',
       'ethnicity1', 'ethnicity2', 'ethnicity3', 'state_ppvi', 'state_npvi',
       'state_employ1', 'state_employ2', 'state_employ3', 'state_payroll1',
       'state_payroll2', 'state_payroll3', 'state_ethnicity1',
       'state_ethnicity2', 'state_ethnicity3', 'issue_lower', 'issue_ch',
       'h_committee_tp1', 'h_committee_tp2', 'h_committee_tp3',
       's_committee_tp1', 's_committee_tp2', 's_committee_tp3',
       'party_tp_type', 'party_tp_score', 'party_tp', 'industry_tp1',
       'industry_tp2', 'industry_tp3', 'ethnicity_tp1', 'ethnicity_tp2',
       'ethnicity_tp3', '国际压力值'],
      dtype='object')

### 4.2 加载政党议题偏好数据

In [55]:
# party_tp_pd = pd.read_excel("./input/议题政党偏好.xlsx", sheet_name="Sheet1")
# party_tp_pd["英文名称"] = party_tp_pd["英文名称"].apply(lambda x:x.lower())
# party_tp_melted = party_tp_pd.melt(
#     id_vars=["英文名称", "参议院委员会", "众议院委员会"], 
#     var_name="congress", 
#     value_name="party_tp"
# )
# party_tp_melted[['party_tp_value', 'party_tp_score']] = party_tp_melted['party_tp'].str.split('+', expand=True)
# party_tp_melted['party_tp_score'] = party_tp_melted['party_tp_score'].fillna(0)

# train_df = train_df.merge(
#     party_tp_melted, 
#     left_on=["issue_lower", "congress"], 
#     right_on=["英文名称", "congress"], 
#     how="left"
# )

# print(f"合并政党偏好数据后的规模: {train_df.shape}")

### 4.3 加载国际压力值数据

In [56]:
# intern_pressure_pd = pd.read_excel("./input/议题国际压力值.xlsx", sheet_name="Sheet1")
# intern_pressure_pd["英文名"] = intern_pressure_pd["英文名"].apply(lambda x:x.lower())
# intern_pressure_pd = intern_pressure_pd.drop("政策领域", axis=1)
# intern_pressure_pd = intern_pressure_pd.rename(columns={"国际压力值": "topic_ip"})

# train_df = train_df.merge(
#     intern_pressure_pd, 
#     left_on=["issue_lower"], 
#     right_on=["英文名"], 
#     how="left"
# )
# print(f"合并国际压力值数据后的规模: {train_df.shape}")

### 4.4 加载产业卷入度数据

In [57]:
# industry_tp_pd = pd.read_excel("./input/议题产业卷入度-三组数据.xlsx",  sheet_name="Sheet1")
# industry_tp_pd["英文名称"] = industry_tp_pd["英文名称"].apply(lambda x:x.lower())
# industry_tp_pd = industry_tp_pd.drop(["政策领域", '118(不带information)', '117(不带information)',
#        '116(不带information)', '115(不带information)',  '118(带information)',
#         '117(带information)',  '116(带information)',  '115(带information)'], axis=1)
# industry_tp_melted = industry_tp_pd.melt(
#     id_vars=["英文名称"], 
#     var_name="congress", 
#     value_name="industry_tp"
# )

# train_df = train_df.merge(
#     industry_tp_melted, 
#     left_on=["issue_lower", "congress"], 
#     right_on=["英文名称", "congress"], 
#     how="left"
# )
# print(f"合并产业卷入度数据后的规模: {train_df.shape}")

### 4.5 加载国会热度数据

In [58]:
# congress_tp_pd = pd.read_excel("./input/议题国会热度.xlsx", sheet_name="ByMonth")
# congress_tp_pd.columns = map(str.lower, congress_tp_pd.columns)
# congress_tp_pd['date'] = congress_tp_pd['date'].apply(
#     lambda x: x.replace(day=1)
# )
# congress_tp_pd = congress_tp_pd.melt(
#     id_vars=["date"], 
#     var_name="issue_lower", 
#     value_name="congress_tp"
# )

# train_df = train_df.merge(
#     congress_tp_pd, 
#     left_on=["date", "issue_lower"], 
#     right_on=["date", "issue_lower"], 
#     how="left"
# )
# print(f"合并国会热度数据后的规模: {train_df.shape}")

### 4.6 加载白宫热度数据

In [59]:
# whitehouse_tp = pd.read_excel("./input/议题白宫热度.xlsx", sheet_name="ByMonth")
# whitehouse_tp.columns = map(str.lower, whitehouse_tp.columns)
# whitehouse_tp['date'] = whitehouse_tp['date'].apply(
#     lambda x: x.replace(day=1)
# )
# whitehouse_tp = whitehouse_tp.melt(
#     id_vars=["date"], 
#     var_name="issue_lower", 
#     value_name="whitehouse_tp"
# )

# train_df = train_df.merge(
#     whitehouse_tp, 
#     left_on=["date", "issue_lower"], 
#     right_on=["date", "issue_lower"], 
#     how="left"
# )
# print(f"合并白宫热度数据后的规模: {train_df.shape}")

### 4.7 加载新闻热度数据

In [60]:
# net_tp_pd = pd.read_excel("./input/议题新闻热度.xlsx", sheet_name="Sheet1")
# net_tp_pd.columns = map(str.lower, net_tp_pd.columns)
# net_tp_pd["month"] = net_tp_pd["month"].apply(
#     lambda x: datetime.strptime(str(x), "%Y%m").date()
# )
# net_tp_pd["month"] = pd.to_datetime(net_tp_pd["month"])
# net_tp_pd = net_tp_pd.melt(
#     id_vars=["month"], 
#     var_name="issue_lower", 
#     value_name="net_tp"
# )

# train_df = train_df.merge(
#     net_tp_pd, 
#     left_on=["date", "issue_lower"], 
#     right_on=["month", "issue_lower"], 
#     how="left"
# )
# print(f"合并新闻热度数据后的规模: {train_df.shape}")

### 4.8 加载智库热度数据

In [61]:
# thinktant_tp_pd = pd.read_excel("./input/议题智库热度.xlsx", sheet_name="Sheet1")
# thinktant_tp_pd.columns = map(str.lower, thinktant_tp_pd.columns)
# thinktant_tp_pd["month"] = thinktant_tp_pd["month"].apply(
#     lambda x: datetime.strptime(str(x), "%Y%m").date()
# )
# thinktant_tp_pd["month"] = pd.to_datetime(thinktant_tp_pd["month"])
# thinktant_tp_pd = thinktant_tp_pd.melt(
#     id_vars=["month"], 
#     var_name="issue_lower", 
#     value_name="thinktant_tp"
# )

# train_df = train_df.merge(
#     thinktant_tp_pd, 
#     left_on=["date", "issue_lower"], 
#     right_on=["month", "issue_lower"], 
#     how="left"
# )
# print(f"合并智库热度数据后的规模: {train_df.shape}")

### 4.9 加载族裔卷入度数据

In [62]:
zhuyi_issue_pd = pd.read_excel("./input/族裔卷入度.xlsx", sheet_name="count")
zhuyi_issue_pd["issue"] = zhuyi_issue_pd["issue"].apply(lambda x:x.lower())
zhuyi_issue_melted = zhuyi_issue_pd.melt(id_vars=["issue"], var_name="category", value_name="value")
zhuyi_issue_melted[['race', 'congress']] = zhuyi_issue_melted['category'].str.extract(r'([A-Za-z\s]+)(\d+)', expand=True)
zhuyi_issue_final = zhuyi_issue_melted.pivot(index=['issue', 'congress'], columns='race', values='value').reset_index()
zhuyi_issue_final = zhuyi_issue_final[['issue', 'congress', 'White', 'Hispanic OR Latino', 'Black OR African American', 
                     'American Indian AND Alaska Native', 'Asian', 'Native Hawaiian OR Pacific Islander']]
zhuyi_issue_final = zhuyi_issue_final.rename(columns={"issue": "issue_lower"})
zhuyi_issue_final["congress"] = zhuyi_issue_final["congress"].apply(int)

train_df = train_df.merge(
    zhuyi_issue_final, 
    left_on=["congress", "issue_lower"], 
    right_on=["congress", "issue_lower"], 
    how="left"
)
print(f"合并族裔卷入度数据后的规模: {train_df.shape}")

合并族裔卷入度数据后的规模: (68640, 68)


### 4.10 pvi数据的再匹配

In [63]:
train_df[['congress','state_code','district']].head(3)

,congress,state_code,district
0,116,UT,99
1,116,MO,4
2,115,CA,3


In [64]:
df_pvi = pd.read_excel(DATA_DIR / 'pvi-full-new.xlsx', sheet_name='full')
# 提取PVI中的字母部分为ppvi，数字部分为npvi
df_pvi['ppvi'] = df_pvi['PVI'].str.extract(r'([A-Za-z]+)')
df_pvi['npvi'] = df_pvi['PVI'].str.extract(r'(\d+)')

# 2. 仅保留df_pvi中用于匹配和更新的必要列，使用copy()避免警告
df_pvi_sub = df_pvi[['congress', 'postal', 'district', 'ppvi', 'npvi']].copy()


In [65]:
# 1. 先处理df_pvi_sub中的district列，将"AL"替换为"At Large"
df_pvi_sub = df_pvi_sub.copy()
df_pvi_sub['district'] = df_pvi_sub['district'].replace('AL', 'At Large')

# 2. 处理缺失值和数据类型统一
# 删除congress为NaN的行
df_pvi_sub = df_pvi_sub.dropna(subset=['congress'])

# 统一数据类型
df_pvi_sub['congress'] = df_pvi_sub['congress'].astype(int)
train_df['congress'] = train_df['congress'].astype(int)

# 统一字符串格式
df_pvi_sub['postal'] = df_pvi_sub['postal'].astype(str).str.strip().str.upper()
train_df['state_code'] = train_df['state_code'].astype(str).str.strip().str.upper()
df_pvi_sub['district'] = df_pvi_sub['district'].astype(str).str.strip()
train_df['district'] = train_df['district'].astype(str).str.strip()

print(f"\n处理后df_pvi_sub的规模: {df_pvi_sub.shape}")
print(f"train_df的规模: {train_df.shape}")

# 3. 合并前备份原有的ppvi和npvi值用于比较
train_df_before = train_df.copy()
original_ppvi_missing = train_df['ppvi'].isna().sum()
original_npvi_missing = train_df['npvi'].isna().sum()

print(f"\n合并前train_df中:")
print(f"ppvi缺失值数量: {original_ppvi_missing}")
print(f"npvi缺失值数量: {original_npvi_missing}")

# 4. 执行左连接合并，更新ppvi和npvi
train_df = train_df.merge(
    df_pvi_sub[['congress', 'postal', 'district', 'ppvi', 'npvi']],
    left_on=['congress', 'state_code', 'district'],
    right_on=['congress', 'postal', 'district'],
    how='left',
    suffixes=('', '_new')
)

# 5. 用新的ppvi和npvi值更新原有值
# 首先用新值覆盖原值（包括非空值）
train_df['ppvi'] = train_df['ppvi_new']
train_df['npvi'] = train_df['npvi_new']

# 6. 删除合并产生的多余列
train_df = train_df.drop(columns=['ppvi_new', 'npvi_new', 'postal'])

# 7. 计算匹配统计
after_ppvi_missing = train_df['ppvi'].isna().sum()
after_npvi_missing = train_df['npvi'].isna().sum()

matched_ppvi = original_ppvi_missing - after_ppvi_missing + (len(train_df) - original_ppvi_missing)
matched_npvi = original_npvi_missing - after_npvi_missing + (len(train_df) - original_npvi_missing)

print(f"\n=== 合并结果统计 ===")
print(f"train_df总行数: {len(train_df)}")
print(f"合并后ppvi缺失值数量: {after_ppvi_missing}")
print(f"合并后npvi缺失值数量: {after_npvi_missing}")
print(f"ppvi成功匹配的数量: {len(train_df) - after_ppvi_missing}")
print(f"npvi成功匹配的数量: {len(train_df) - after_npvi_missing}")
print(f"ppvi未匹配的数量: {after_ppvi_missing}")
print(f"npvi未匹配的数量: {after_npvi_missing}")

# # 8. 详细分析未匹配的数据
# unmatched_data = train_df[train_df['ppvi'].isna()]
# if len(unmatched_data) > 0:
#     print(f"\n=== 未匹配数据分析 ===")
#     print(f"未匹配数据总数: {len(unmatched_data)}")
#     print("\n未匹配数据的congress分布:")
#     print(unmatched_data['congress'].value_counts().sort_index())
#     print("\n未匹配数据的state_code分布:")
#     print(unmatched_data['state_code'].value_counts())
#     print("\n未匹配数据的district分布:")
#     print(unmatched_data['district'].value_counts())
    
#     # 显示几个未匹配的样例
#     print("\n未匹配数据样例:")
#     print(unmatched_data[['congress', 'state_code', 'district', 'ppvi', 'npvi']].head())
# else:
#     print(f"\n所有数据都成功匹配!")

# # 9. 验证匹配效果
# print(f"\n=== 匹配效果验证 ===")
# print(f"匹配率: {((len(train_df) - after_ppvi_missing) / len(train_df) * 100):.2f}%")


处理后df_pvi_sub的规模: (2970, 5)
train_df的规模: (68640, 68)

合并前train_df中:
ppvi缺失值数量: 1280
npvi缺失值数量: 3008

=== 合并结果统计 ===
train_df总行数: 68640
合并后ppvi缺失值数量: 736
合并后npvi缺失值数量: 2432
ppvi成功匹配的数量: 67904
npvi成功匹配的数量: 66208
ppvi未匹配的数量: 736
npvi未匹配的数量: 2432


## 特征筛选与数据存储

In [66]:
# 修正列名
train_df = train_df.rename(columns={
    "参议院委员会": "s_committee_tp",
    "众议院委员会":"h_committee_tp"
})

In [67]:
# 选择最终所需的特征列
train_df = train_df[[
    "issue", "mid", "full_name", "chamber",
    "age", "party", "state_code", "gender", "congress",
    "committee_el", "com_post",
    "district",    # 补充选区的数据
    "ideology",    # 补充意识形态的数据
    "seniority",   # 资历
    
    # 时间信息
    # "cur_year", "cur_month",
    
    # 选区层面
    "ppvi", "npvi",
    "employ1", "employ2", "employ3", 
    "payroll1", "payroll2", "payroll3",
    "ethnicity1", "ethnicity2", "ethnicity3",

    # # 州层面
    'state_ppvi', 'state_npvi',
    # 'state_employ1', 'state_employ2', 'state_employ3',
    # 'state_payroll1','state_payroll2', 'state_payroll3',
    # 'state_ethnicity1','state_ethnicity2', 'state_ethnicity3',

    # 政党议题偏好、国际压力值、产业卷入度
    'h_committee_tp1', 'h_committee_tp2', 'h_committee_tp3',
    's_committee_tp1', 's_committee_tp2', 's_committee_tp3',
    'party_tp_type', 'party_tp_score', 'party_tp', 'industry_tp1',
    'industry_tp2', 'industry_tp3', 'ethnicity_tp1', 'ethnicity_tp2',
    'ethnicity_tp3', '国际压力值',

    # 热度数据
    # 'congress_tp', 'whitehouse_tp','net_tp','thinktant_tp',
     
     
    # "White", "Hispanic OR Latino",
    # "Black OR African American", "American Indian AND Alaska Native",
    # "Asian", "Native Hawaiian OR Pacific Islander",
     
    # 标签
    "label"
]]

In [68]:
# 保存最终数据到CSV文件
output_file = TRAIN_DATA_DIR / "legis_train_data_with_feature_更新test.csv"
train_df.to_csv(output_file, index=False)

In [69]:
# 测试加载保存的数据
test_df = pd.read_csv(output_file)
print(f"读取的数据形状: {test_df.shape}")

读取的数据形状: (68640, 44)


In [70]:
test_df.columns

Index(['issue', 'mid', 'full_name', 'chamber', 'age', 'party', 'state_code',
       'gender', 'congress', 'committee_el', 'com_post', 'district',
       'ideology', 'seniority', 'ppvi', 'npvi', 'employ1', 'employ2',
       'employ3', 'payroll1', 'payroll2', 'payroll3', 'ethnicity1',
       'ethnicity2', 'ethnicity3', 'state_ppvi', 'state_npvi',
       'h_committee_tp1', 'h_committee_tp2', 'h_committee_tp3',
       's_committee_tp1', 's_committee_tp2', 's_committee_tp3',
       'party_tp_type', 'party_tp_score', 'party_tp', 'industry_tp1',
       'industry_tp2', 'industry_tp3', 'ethnicity_tp1', 'ethnicity_tp2',
       'ethnicity_tp3', '国际压力值', 'label'],
      dtype='object')